# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,avarua,-21.2078,-159.7750,26.03,83,75,10.29,CK,1702331664
1,1,port-aux-francais,-49.3500,70.2167,5.65,98,99,10.34,TF,1702331700
2,2,tindouf,27.6711,-8.1474,17.25,27,80,3.09,DZ,1702331735
3,3,tiksi,71.6872,128.8694,-22.19,100,100,3.85,RU,1702331735
4,4,hawaiian paradise park,19.5933,-154.9731,25.57,94,75,1.54,US,1702331736


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:


# Configure the map plot
map_plot= city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=600,
    frame_height=500,
    size="Humidity",
    color="City")

# Display the map
#hvplot.save(map_plot, "../output_data/Fig1.png")
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cond_df = city_data_df.loc[((city_data_df["Max Temp"] <= 30.0) & (city_data_df["Max Temp"] >= 20.0))\
                                 & (city_data_df["Cloudiness"] == 0),:]


# Drop any rows with null values
ideal_cond_df = ideal_cond_df.dropna(how="any")

# Display sample data
ideal_cond_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
159,159,touros,-5.1989,-35.4608,26.68,75,0,6.19,BR,1702331676
162,162,pisco,-13.7000,-76.2167,23.03,73,0,8.75,PE,1702331745
169,169,manaure,11.7751,-72.4445,29.96,60,0,8.79,CO,1702331787
218,218,cabo san lucas,22.8909,-109.9124,28.12,60,0,2.07,MX,1702331578
230,230,geraldton,-28.7667,114.6000,22.23,26,0,5.14,AU,1702331780


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cond_df.iloc[:,[1,8,2,3,5]].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''
# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
159,touros,BR,-5.1989,-35.4608,75,
162,pisco,PE,-13.7000,-76.2167,73,
169,manaure,CO,11.7751,-72.4445,60,
218,cabo san lucas,MX,22.8909,-109.9124,60,
230,geraldton,AU,-28.7667,114.6000,26,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {'categories':'accommodation.hotel',\
         'filter':'',\
         'bias':'',\
         'apiKey':geoapify_key,\
         'limit':1}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon = row['Lng']
    lat = row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
#         # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
#     # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# # Display sample data
hotel_df.head()

Starting hotel search
touros - nearest hotel: Pousada Atlântico
pisco - nearest hotel: La Portada
manaure - nearest hotel: hotel palaima
cabo san lucas - nearest hotel: Comfort Rooms
geraldton - nearest hotel: Ocean Centre Hotel
new shagunnu - nearest hotel: No hotel found
keffi - nearest hotel: No hotel found
makurdi - nearest hotel: Benue Hotel and Resort
tahoua - nearest hotel: Hotel Bungalow
mount isa - nearest hotel: Ibis Styles
nampula - nearest hotel: Hotel Lurio
douentza - nearest hotel: No hotel found
maragogi - nearest hotel: Pousada Mariluz
carnarvon - nearest hotel: No hotel found
piton saint-leu - nearest hotel: Appartement Piton Saint Leu
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
tazacorte - nearest hotel: App Leyma
long beach - nearest hotel: Westin


,City,Country,Lat,Lng,Humidity,Hotel Name
159,touros,BR,-5.1989,-35.4608,75,Pousada Atlântico
162,pisco,PE,-13.7000,-76.2167,73,La Portada
169,manaure,CO,11.7751,-72.4445,60,hotel palaima
218,cabo san lucas,MX,22.8909,-109.9124,60,Comfort Rooms
230,geraldton,AU,-28.7667,114.6000,26,Ocean Centre Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
# Configure the map plot
map_plot2 = hotel_df.hvplot.points('Lng',
            'Lat',
            geo=True, 
            tiles = 'OSM', 
            frame_width = 700, 
            frame_height = 500, 
            color='City',
            alpha=0.6,
            hover=True,
            hover_cols='all',
            use_index=False,
            s='Humidity',
            scale= 1.2)

# Display the map
map_plot2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)